In [2]:
import numpy as np
import random
import pickle

In [15]:
cd first

/content/drive/My Drive/Colab Notebooks/first


In [16]:
# loading training set features
f = open("Datasets/train_set_features.pkl", "rb")
train_set_features2 = pickle.load(f)
f.close()

# reducing feature vector length 
features_STDs = np.std(a=train_set_features2, axis=0)
train_set_features = train_set_features2[:, features_STDs > 52.3]

# changing the range of data between 0 and 1
train_set_features = np.divide(train_set_features, train_set_features.max())

# loading training set labels
f = open("Datasets/train_set_labels.pkl", "rb")
train_set_labels = pickle.load(f)
f.close()

# ------------
# loading test set features
f = open("Datasets/test_set_features.pkl", "rb")
test_set_features2 = pickle.load(f)
f.close()

# reducing feature vector length 
features_STDs = np.std(a=test_set_features2, axis=0)
test_set_features = test_set_features2[:, features_STDs > 48]

# changing the range of data between 0 and 1
test_set_features = np.divide(test_set_features, test_set_features.max())

# loading test set labels
f = open("Datasets/test_set_labels.pkl", "rb")
test_set_labels = pickle.load(f)
f.close()

# ------------
# preparing our training and test sets - joining datasets and lables
train_set = []
test_set = []

for i in range(len(train_set_features)):
    label = np.array([0,0,0,0])
    label[int(train_set_labels[i])] = 1
    label = label.reshape(4,1)
    train_set.append((train_set_features[i].reshape(102,1), label))
    

for i in range(len(test_set_features)):
    label = np.array([0,0,0,0])
    label[int(test_set_labels[i])] = 1
    label = label.reshape(4,1)
    test_set.append((test_set_features[i].reshape(102,1), label))

# shuffle
random.shuffle(train_set)
random.shuffle(test_set)

# print size
print(len(train_set)) #1962
print(len(test_set)) #662


1962
662


In [21]:
train_set[0]

(array([[5.06720850e-01],
        [2.14442013e-01],
        [3.51672398e-01],
        [3.47921225e-01],
        [2.67896218e-01],
        [1.25664270e-01],
        [1.32854017e-01],
        [1.12222570e-01],
        [6.81462957e-02],
        [5.81431697e-02],
        [1.75367302e-01],
        [2.68834011e-02],
        [3.53235386e-02],
        [3.43857455e-03],
        [1.81306658e-02],
        [3.87621132e-02],
        [4.37636761e-03],
        [8.12753986e-03],
        [9.37793060e-04],
        [2.81337918e-03],
        [1.78180681e-02],
        [1.37542982e-02],
        [6.25195374e-04],
        [3.43857455e-03],
        [1.12535167e-02],
        [5.93935605e-03],
        [3.12597687e-04],
        [6.25195374e-04],
        [0.00000000e+00],
        [8.44013754e-03],
        [0.00000000e+00],
        [9.37793060e-04],
        [0.00000000e+00],
        [1.25039075e-03],
        [0.00000000e+00],
        [3.75117224e-03],
        [1.56298843e-03],
        [0.00000000e+00],
        [0.0